In [1]:
import os
os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is C:\Users\Gubbz\Documents\NSS\NSS_Projects\accre-pumpkin-pie


In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [3]:
jobs = pd.read_csv("data/fullsample.csv")
jobs.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [4]:
jobs_filtered = jobs[(jobs['STATE'] == 'COMPLETED') | (jobs['STATE'] == 'FAILED')].copy()

In [5]:
jobs_filtered['END'] = jobs_filtered['END'].str.replace('T', ' ')
jobs_filtered['END'] = pd.to_datetime(jobs_filtered['END'], format = "%Y-%m-%d %H:%M:%S")

In [6]:
jobs_filtered['BEGIN'] = jobs_filtered['BEGIN'].str.replace('T', ' ')
jobs_filtered['BEGIN'] = pd.to_datetime(jobs_filtered['BEGIN'], format = "%Y-%m-%d %H:%M:%S")
jobs_filtered.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [70]:
typical_day_count = jobs_filtered.groupby([jobs_filtered['END'].dt.date, jobs_filtered['END'].dt.hour]).agg({'JOBID':'count'})
typical_day_count.index.names = ['Date', 'Hour']

In [72]:
typical_day_count

JOBID
Date       Hour       
2020-10-01 0       421
           1      1080
           2      1137
           3      1018
           4      1244
...                ...
2021-10-07 16        8
           17        8
           18        8
           19        8
           20        6

[8922 rows x 1 columns]

In [38]:
typical_day_count_END = jobs_filtered.groupby([jobs_filtered['END'].dt.date, jobs_filtered['END'].dt.hour]).agg({'JOBID':'count'})
typical_day_count_END.index.names = ['Date', 'Hour']

In [39]:
typical_day_count_END = typical_day_count_END.reset_index()
typical_day_count_END = typical_day_count_END.rename(columns={'JOBID':'JOBS_FINISHED'})
typical_day_count_END

,Date,Hour,JOBS_FINISHED
0,2020-10-01,0,421
1,2020-10-01,1,1080
2,2020-10-01,2,1137
3,2020-10-01,3,1018
4,2020-10-01,4,1244
...,...,...,...
8917,2021-10-07,16,8
8918,2021-10-07,17,8
8919,2021-10-07,18,8
8920,2021-10-07,19,8


In [52]:
typical_day_count_BEGIN = jobs_filtered.groupby([jobs_filtered['BEGIN'].dt.date, jobs_filtered['BEGIN'].dt.hour]).agg({'JOBID':'count'})
typical_day_count_BEGIN.index.names = ['Date', 'Hour']

In [53]:
typical_day_count_BEGIN = typical_day_count_BEGIN.reset_index()
typical_day_count_BEGIN = typical_day_count_BEGIN.rename(columns={'JOBID':'JOBS_STARTED'})
typical_day_count_BEGIN

,Date,Hour,JOBS_STARTED
0,2020-10-01,0,773
1,2020-10-01,1,1135
2,2020-10-01,2,1038
3,2020-10-01,3,1107
4,2020-10-01,4,1390
...,...,...,...
8916,2021-10-07,16,8
8917,2021-10-07,17,8
8918,2021-10-07,18,8
8919,2021-10-07,19,8


In [54]:
merged_df = pd.merge(typical_day_count_END, typical_day_count_BEGIN, on=['Date','Hour'], suffixes = ['_Completed', '_Started'])
merged_df

,Date,Hour,JOBS_FINISHED,JOBS_STARTED
0,2020-10-01,0,421,773
1,2020-10-01,1,1080,1135
2,2020-10-01,2,1137,1038
3,2020-10-01,3,1018,1107
4,2020-10-01,4,1244,1390
...,...,...,...,...
8916,2021-10-07,16,8,8
8917,2021-10-07,17,8,8
8918,2021-10-07,18,8,8
8919,2021-10-07,19,8,8


In [58]:
merged_df['JOBS_FINISHED'].corr(merged_df['JOBS_STARTED'])

0.969025626714075

In [60]:
merged_df.describe()

,Hour,JOBS_FINISHED,JOBS_STARTED
count,8921.000000,8921.000000,8921.000000
mean,11.498711,827.132384,827.132608
std,6.921551,1187.339711,1210.425794
min,0.000000,2.000000,3.000000
25%,5.000000,250.000000,207.000000
50%,12.000000,488.000000,474.000000
75%,17.000000,908.000000,951.000000
max,23.000000,13346.000000,13671.000000


In [ ]:
date_high = pd.to_datetime('2020-12-04').date()
high_failures_day_END_df = jobs_filtered[jobs_filtered['END'].dt.date == date_high]

In [ ]:
date_high = pd.to_datetime('2020-12-04').date()
high_failures_day_BEGIN_df = jobs_filtered[jobs_filtered['BEGIN'].dt.date == date_high]

In [ ]:
display(high_failures_day_END_df.shape)
display(high_failures_day_BEGIN_df.shape)

In [ ]:
date_low = pd.to_datetime('2021-03-23').date()
low_failures_day_END_df = jobs_filtered[jobs_filtered['END'].dt.date == date_low]

In [ ]:
date_low = pd.to_datetime('2021-03-23').date()
low_failures_day_BEGIN_df = jobs_filtered[jobs_filtered['BEGIN'].dt.date == date_low]

In [ ]:
display(low_failures_day_END_df.shape)
display(low_failures_day_BEGIN_df.shape)

In [ ]:
jobs_filtered['END'].corr(jobs_filtered['BEGIN'])

In [74]:
typical_day_count.loc[datetime.date(2020,12,4)]

,JOBID
Hour,
0,1876
1,1676
2,722
3,308
4,1431
5,278
6,292
7,535
8,586
